In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from dataloader import *

In [2]:
class Model(nn.Module):
    def __init__(self,num_layers):
        super(Model,self).__init__()
        self.model1 = nn.ModuleList([nn.Conv2d(1,1,3,padding=1,bias=False) for i in range(num_layers)])
        self.model2 = nn.ModuleList([nn.Conv2d(1,1,3,padding=1,bias=False) for i in range(num_layers)])
        self.model3 = nn.ModuleList([nn.Conv2d(1,1,3,padding=1,bias=False) for i in range(num_layers)])
        self.model4 = nn.ModuleList([nn.Conv2d(1,1,3,padding=1,bias=False) for i in range(num_layers)])
#         self.model = nn.Sequential(self.model1)
    def forward(self,x):
#         print(x)
        x1 = reduce(lambda acc, el: el(acc), self.model1, x)
        x2 = reduce(lambda acc, el: el(acc), self.model2, x)
        x3 = reduce(lambda acc, el: el(acc), self.model3, x)
        x4 = reduce(lambda acc, el: el(acc), self.model4, x)
        
        x = torch.cat((x1,x2,x3,x4),1)
        return x

In [3]:
torch.load('./Model/model.pt')

Model(
  (model1): ModuleList(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (model2): ModuleList(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (model3): ModuleList(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (model4): ModuleList(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Conv2d(1, 1, kernel_size=(

In [5]:
num_workers = 1
batch_size = 1
testData = Ipde('./data/128/')


testDataloader = DataLoader(testData,num_workers=num_workers,batch_size=batch_size)

testIter = iter(testDataloader)
test = testIter.next()

In [7]:
model = Model(3)

In [8]:
model.eval()
counter = 0
tot_error = 0.0
for i,(parameter,data,target) in enumerate(testDataloader):
    if(torch.cuda.is_available()):
        data = data.type(torch.FloatTensor).cuda()
        parameter = parameter.type(torch.FloatTensor).cuda()
        target = target.type(torch.FloatTensor).cuda()
    else:
        data = data.type(torch.FloatTensor)
        parameter = parameter.type(torch.FloatTensor)
        target = target.type(torch.FloatTensor)
        
    output = phi_input(data,parameter,0.1)
    data = data.cpu().detach().numpy()
    target = target.cpu().detach().numpy()
    output = output.cpu()
    output = output.detach().numpy()
    output = output.squeeze(0)
    image = output.squeeze(0)
    input_image = data.squeeze(0).squeeze(0)
    target_image = target.squeeze(0).squeeze(0)
    plt.figure(i)
    plt.subplot(1,3,1)
    plt.title('Input image')
    plt.axis('off')
    plt.imshow(input_image)
    plt.subplot(1,3,2)
    plt.title('Target image')
    plt.axis('off')
    plt.imshow(target_image)
    plt.subplot(1,3,3)
    plt.title('Output image')
    plt.axis('off')
    plt.imshow(image)
    counter+=1
    if(counter == 20):
        break
    error = np.sqrt(np.mean((output - data)**2))
#     print('root mean square error',error)
    tot_error += error
avg_error = tot_error/len(testDataloader)
print(avg_error)

NameError: name 'phi_input' is not defined